- 时间序列预测任务示例。
- 此notebook不在项目根目录下，使用os模块更改路径为上层目录（根目录）。

In [ ]:
import os
os.chdir("../")

**Step 1. Choose a dataset (`framework.forecasting.forecasting_datamodule`)**

- 本项目的数据集都需要使用`LightningDataModule`加载。一个`LightningDataModule`实例包括了训练、验证、测试所需的所有数据加载器，无需重复分别实现。
- 使用项目自带的`FrecastingDataModule`可以方便地将原始数据转换为用于序列预测任务的`LightningDataModule`。
- 本项目自带的数据集位于`data/`路径下。作为一个简单的示例，此处选择交通预测数据集：北京地铁进站客流数据，共276个站点，每10分钟的数据聚合为1个时间步。

In [ ]:
from utils.create_datamodule.forecasting_datamodule import ForecastingDataModule

datamodule = ForecastingDataModule(
    csv_file_path="data/BJinflow/in_10min_trans.csv",
    stride=1,
    input_length=24,
    output_length=24,
    batch_size=128,
    train_val_test_split=(0.6, 0.2, 0.2),
    num_workers=7,
    normalization="01",
)

**Step 2. Choose a model (`models.forecasting_models`)**

- 本项目的模型根据任务类别进行分类。不同任务的模型已经各自实现了所需功能。
- 本项目自带的模型位于`models`模块。
- 作为一个简单的示例，此处选择简单的LSTM模型。

In [ ]:
import models

model = models.forecasting_models.LSTM(
    input_size=276,  # here we know the input size is 276
    hidden_size=128,
    num_layers=2,
    bidirectional=True,
    lr=1e-3,
    max_epochs=20,
    loss_type="mse",
)

- 本项目使用`Lightning`框架，模型构建后，自动选择可用加速器：CPU、CUDA、MPS等等。

**Step 3. Model trains itself with the datamodule**

- 只需要简单调用`model.run_training()`或`model.fit()`即可开始训练。
任务所需的其他所有功能已为你实现：

    1. 自动打印超参数、模型结构、训练过程等任何信息
    2. 自动启动Tensorboard（在浏览器中打开`http://localhost:6060`)
    3. 自动保存模型图（在`Tensorboard-Graph`中查看）
    4. 自动计算并保存任务相关指标（在`Tensorboard-Scalar`中查看）
    5. 自动完成并保存任务相关可视化（在`Tensorboard-Image`中查看）
    6. 自动保存超参数及其对应指标（在`Tensorboard-Hparams`中查看）
    7. 基于验证集损失自动保存checkpoint(在`lightning_logs/`路径下)
    8. 训练完成后自动从checkpoint加载训练过程中的最佳模型

In [ ]:
model.fit(datamodule)

- 完成训练后，最优权重已自动从checkpoint加载。同样地，只需简单调用`model.run_test()`或`model.test()`即可开始模型测试。

In [ ]:
model.test(datamodule)

**Step 4. Model predicts**

- 与标准的`nn.Module`模块一样，你可以直接调用`model(input)`或`model.forward(input)`来执行前向传播。

In [ ]:
from utils.visualization import SeriesPlotter

# 从datamodule取出一个样本：
sample = datamodule.test_dataset[0]
x, y = sample
predicted = model(x)
# 查看预测结果：
SeriesPlotter.plot_and_show({"original": y, "predicted": predicted})